# 混合搜索

LangChain 中的标准搜索是通过向量相似性完成的。然而，许多 [向量存储](/docs/integrations/vectorstores/) 实现（Astra DB、ElasticSearch、Neo4J、AzureSearch、Qdrant 等）还支持更高级的搜索，它结合了向量相似性搜索和其他搜索技术（全文、BM25 等）。这通常被称为“混合”搜索。

**步骤 1：确保您使用的向量存储支持混合搜索**

目前，LangChain 中没有统一的方法来执行混合搜索。每种向量存储可能有自己的方法。这通常会作为在进行 `similarity_search` 时传递的关键字参数暴露出来。

通过阅读文档或源代码，弄清楚您使用的向量存储是否支持混合搜索，如果支持，如何使用它。

**步骤 2：为链添加该参数作为可配置字段**

这将允许您在运行时轻松调用链并配置任何相关的标志。有关配置的更多信息，请参阅[此文档](/docs/how_to/configure)。

**步骤 3：使用该可配置字段调用链**

现在，在运行时，您可以使用该可配置字段调用此链。

## 代码示例

让我们通过一个具体的代码示例来看看这是什么样子。在本示例中，我们将使用 Astra DB 的 Cassandra/CQL 接口。

安装以下 Python 包：

In [ ]:
!pip install "cassio>=0.1.7"

获取[连接密钥](https://docs.datastax.com/en/astra/astra-db-vector/get-started/quickstart.html)。

初始化 cassio：

In [ ]:
import cassio

cassio.init(
    database_id="Your database ID",
    token="Your application token",
    keyspace="Your key space",
)

使用标准的[索引分析器](https://docs.datastax.com/en/astra/astra-db-vector/cql/use-analyzers-with-cql.html)创建 Cassandra VectorStore。索引分析器对于启用词语匹配是必需的。

In [ ]:
from cassio.table.cql import STANDARD_ANALYZER
from langchain_community.vectorstores import Cassandra
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = Cassandra(
    embedding=embeddings,
    table_name="test_hybrid",
    body_index_options=[STANDARD_ANALYZER],
    session=None,
    keyspace=None,
)

vectorstore.add_texts(
    [
        "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",
    ]
)

如果我们进行标准的相似性搜索，我们会得到所有文档：

In [ ]:
vectorstore.as_retriever().invoke("What city did I visit last?")

[Document(page_content='In 2022, I visited New York'),
Document(page_content='In 2023, I visited Paris'),
Document(page_content='In 2021, I visited New Orleans')]

Astra DB vectorstore 的 `body_search` 参数可用于过滤搜索词 `new`。

In [ ]:
vectorstore.as_retriever(search_kwargs={"body_search": "new"}).invoke(
    "What city did I visit last?"
)

[Document(page_content='In 2022, I visited New York'),
Document(page_content='In 2021, I visited New Orleans')]

现在，我们可以创建用于问答的链了

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    ConfigurableField,
    RunnablePassthrough,
)
from langchain_openai import ChatOpenAI

这是一个基础的问答链设置。

In [ ]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

retriever = vectorstore.as_retriever()

这里我们将检索器标记为具有可配置字段。所有向量存储检索器都有一个名为 `search_kwargs` 的字段。它就是一个字典，包含向量存储特定的字段。

In [ ]:
configurable_retriever = retriever.configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

现在我们可以使用我们可配置的检索器来创建链了。

In [ ]:
chain = (
    {"context": configurable_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("What city did I visit last?")

Paris

现在我们可以用可配置的选项来调用链了。`search_kwargs` 是可配置字段的 id。该值是用于 Astra DB 的搜索 kwargs。

In [ ]:
chain.invoke(
    "What city did I visit last?",
    config={"configurable": {"search_kwargs": {"body_search": "new"}}},
)

New York